In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


"""
Selva - notes
onehot_encode = OneHotEncoder()
print (onehot_encode)

#one hot encoding the categorical values
feature_array = onehot_encode.fit_transform(X[["Pclass","Sex","SibSp","Parch","Cabin","Embarked"]]).toarray()

#labelling the columns with the categorical features
feature_labels = onehot_encode.categories_
#now we have to flaten the list since feature_labels has nested list in it
flattened_label = []
for sublist in feature_labels:
    for items in sublist:
        flattened_label.append(items)

print (flattened_label)
#np.array(feature_labels).ravel()
"""

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


'\nSelva - notes\nonehot_encode = OneHotEncoder()\nprint (onehot_encode)\n\n#one hot encoding the categorical values\nfeature_array = onehot_encode.fit_transform(X[["Pclass","Sex","SibSp","Parch","Cabin","Embarked"]]).toarray()\n\n#labelling the columns with the categorical features\nfeature_labels = onehot_encode.categories_\n#now we have to flaten the list since feature_labels has nested list in it\nflattened_label = []\nfor sublist in feature_labels:\n    for items in sublist:\n        flattened_label.append(items)\n\nprint (flattened_label)\n#np.array(feature_labels).ravel()\n'

### Understanding the Data

In [2]:
import pandas as pd
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
print (train_data.columns)

#splitting the training data into Y (predicted variable) and X 
#(Independent variable - predictor of X)

X = train_data.drop(['Survived'], axis = 1)
Y = train_data['Survived']

print ("Independent variables(X): ", X[:4])
print ("To be predicted variables(Y): ", Y[:4])


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Independent variables(X):     PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S  
1  female  38.0      1      0          PC 17599  71.2833   C85        C  
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S  
3  female  35.0      1      0            113803  53.1000  C123        S  
To be predicted variables(Y):  0    0
1    1
2    1
3    1
Name:

### Feature Selection

-- <font color = 'green'>Finding relevant features in the dataset X which will influence the Y value </font>

In [3]:
#the dtypes gives the data types of each variable
print (X.dtypes)
# from the above we get name,sex, ticket, cabin and embarked as object data types.
#It is clear that name does not need to be one hot encoded as it doesn't result in any
#useful prediction. 
#To confirm embarked, cabin and sex lets perform a group by search to confirm if they 
#can be one hot encoded.

# doing a distinct search in categorical features for one hot encoding.

cabin_count = pd.unique(X['Cabin'])
print ("Cabin, ", X["Cabin"].unique())
#print (len(cabin_count))
#print (len(X))
print ("Sex: ", X["Sex"].unique())
print ("Pclass: ", X["Pclass"].unique()) #categorical feature one hot encode pending
print("SibSp: ", X["SibSp"].unique()) #categorical feature one hot encode pending
print ("Parch: ", X["Parch"].unique())

print (X["Embarked"].unique())

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
Cabin,  [nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34'
 'C104' 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79'
 'E25' 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68'
 'A10' 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23' 'B50' 'A26' 'D48' 'E58'
 '

### One hot encoding

--<font color = 'green'> based on the above code the features Cabin, Sex and Embarked are categorical features, so lets convert them to numerical data with one hot encoding. </font>

In [4]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np

def data_preprocessing(X):
    ##Step 1: One hot encoding categorical features
    onehot_encode = OneHotEncoder() #print (onehot_encode)
    
    feature_array = onehot_encode.fit_transform(X[["Pclass","Sex","SibSp","Parch","Cabin","Embarked"]]).toarray()
    feature_labels = onehot_encode.categories_ #print(feature_labels)
    
    #dropping the encoded features
    X = X.drop(columns = ["Pclass","Sex","SibSp","Parch","Cabin","Embarked"])
    
    flattened_label = []
    for sublist in feature_labels:
        for items in sublist:
            flattened_label.append(items)
    
    #print (flattened_label)
    features = pd.DataFrame(feature_array, columns = flattened_label)
    
    ## Step 2: Scale the numerical features to lie between 0 and 1
    # I am using MinMax scaler.
    
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(X[["Age", "Fare"]])
    scaled_df = pd.DataFrame(scaled_features, columns=["Age", "Fare"])
    X = X.drop(columns=["Age", "Fare"])
    X = pd.concat([X,features,scaled_df], axis=1)
    return X



In [5]:
X_new = data_preprocessing(X)
print(type(X_new))
print (type(Y))

print (X_new.columns)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
Index(['PassengerId',        'Name',      'Ticket',             1,
                   2,             3,      'female',        'male',
                   0,             1,
       ...
                'F4',          'G6',           'T',           nan,
                 'C',           'Q',           'S',           nan,
               'Age',        'Fare'],
      dtype='object', length=176)
